# Lab 3.2.4: GPTQ Quantization

**Module:** 3.2 - Model Quantization & Optimization  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐☆☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the GPTQ algorithm and Hessian-based weight correction
- [ ] Quantize a model using auto-gptq with different group sizes (32/64/128)
- [ ] Compare quality and performance across configurations
- [ ] Load and run pre-quantized GPTQ models from Hugging Face

---

## 📚 Prerequisites

- Completed: Labs 3.2.1-3.2.3
- Library: `auto-gptq` (`pip install auto-gptq`)
- Hardware: CUDA GPU (DGX Spark recommended)

---

## 🌍 Real-World Context

**Why GPTQ is Popular:**
- Published in 2022, became THE standard for 4-bit GPU inference
- Used by TheBloke's 1000+ quantized models on Hugging Face
- Native support in transformers, vLLM, and TensorRT-LLM

**The Magic:** GPTQ uses the Hessian (second derivative) to find which weights are most important, then compensates for quantization errors in a layer-by-layer fashion.

---

## 🧒 ELI5: How GPTQ Works

> **Imagine you're packing a suitcase with a weight limit...**
>
> **Naive approach:** Remove weight from EVERYTHING equally.
> - Problem: Your toothbrush is now too light to use!
>
> **GPTQ approach:**
> 1. **Weigh each item** - figure out which items matter most
> 2. **Pack important items carefully** - keep precision where it counts
> 3. **Redistribute weight** - if you make one thing lighter, make something else heavier to compensate
>
> **In AI terms:**
> 1. Compute the Hessian to find important weights
> 2. Quantize less-important weights more aggressively
> 3. Adjust remaining weights to compensate for errors
>
> The result: 4-bit weights that perform almost like 16-bit!

---

## Part 1: Environment Setup

In [ ]:
import torch
import gc
import time
from pathlib import Path

print("=" * 60)
print("GPTQ Quantization Lab")
print("=" * 60)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Check for auto-gptq
try:
    from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
    HAS_AUTOGPTQ = True
    print(f"\nauto-gptq: Available")
except ImportError:
    HAS_AUTOGPTQ = False
    print(f"\nauto-gptq: Not installed")
    print("  Install with: pip install auto-gptq")

print("=" * 60)

---

## Part 2: Understanding GPTQ Algorithm

GPTQ builds on the Optimal Brain Quantization (OBQ) algorithm.

In [ ]:
# Simplified GPTQ demonstration
import numpy as np
import matplotlib.pyplot as plt

def simple_gptq_quantize(weights, bits=4, group_size=128):
    """
    Simplified GPTQ-style quantization.
    
    Real GPTQ uses:
    1. Hessian to find weight importance
    2. Sequential quantization with error compensation
    3. Group-wise scaling
    
    This demo shows the group-wise concept.
    """
    out_features, in_features = weights.shape
    n_groups = in_features // group_size
    qmax = 2 ** (bits - 1) - 1
    
    # Process each group separately
    quantized = torch.zeros_like(weights, dtype=torch.int8)
    scales = torch.zeros(out_features, n_groups)
    
    for g in range(n_groups):
        start = g * group_size
        end = start + group_size
        group = weights[:, start:end]
        
        # Per-group scale (symmetric)
        max_val = group.abs().amax(dim=1, keepdim=True)
        scale = max_val / qmax
        scale = torch.clamp(scale, min=1e-10)
        
        # Quantize
        q = torch.round(group / scale).clamp(-qmax-1, qmax).to(torch.int8)
        
        quantized[:, start:end] = q
        scales[:, g] = scale.squeeze()
    
    return quantized, scales


def dequantize_gptq(quantized, scales, group_size=128):
    """Dequantize GPTQ-style weights."""
    out_features, in_features = quantized.shape
    n_groups = in_features // group_size
    
    dequantized = torch.zeros(out_features, in_features, dtype=torch.float32)
    
    for g in range(n_groups):
        start = g * group_size
        end = start + group_size
        q = quantized[:, start:end].float()
        scale = scales[:, g:g+1]
        dequantized[:, start:end] = q * scale
    
    return dequantized


# Test with sample weights
torch.manual_seed(42)
weights = torch.randn(256, 1024)  # [out_features, in_features]

print("GPTQ Group-Size Comparison")
print("=" * 60)

for group_size in [32, 64, 128, 256]:
    q, scales = simple_gptq_quantize(weights, bits=4, group_size=group_size)
    deq = dequantize_gptq(q, scales, group_size=group_size)
    error = (weights - deq).abs()
    
    # Memory overhead from scales
    scale_overhead = scales.numel() * 2 / (weights.numel() * 0.5) * 100
    
    print(f"Group size {group_size:>4}: RMSE={error.pow(2).mean().sqrt():.6f}, "
          f"Max error={error.max():.4f}, "
          f"Scale overhead={scale_overhead:.1f}%")

In [ ]:
# Visualize the quality-overhead trade-off

group_sizes = [16, 32, 64, 128, 256, 512]
rmses = []
overheads = []

for gs in group_sizes:
    if 1024 % gs == 0:  # Must divide evenly
        q, scales = simple_gptq_quantize(weights, bits=4, group_size=gs)
        deq = dequantize_gptq(q, scales, group_size=gs)
        rmse = (weights - deq).pow(2).mean().sqrt().item()
        overhead = scales.numel() * 2 / (weights.numel() * 0.5) * 100
        rmses.append(rmse)
        overheads.append(overhead)
    else:
        rmses.append(None)
        overheads.append(None)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Quality vs group size
ax = axes[0]
valid = [(gs, r) for gs, r in zip(group_sizes, rmses) if r is not None]
ax.plot([v[0] for v in valid], [v[1] for v in valid], 'bo-', linewidth=2, markersize=8)
ax.set_xlabel('Group Size')
ax.set_ylabel('RMSE (lower is better)')
ax.set_title('Quantization Quality vs Group Size')
ax.set_xscale('log', base=2)
ax.grid(True, alpha=0.3)

# Overhead vs group size
ax = axes[1]
valid = [(gs, o) for gs, o in zip(group_sizes, overheads) if o is not None]
ax.plot([v[0] for v in valid], [v[1] for v in valid], 'ro-', linewidth=2, markersize=8)
ax.set_xlabel('Group Size')
ax.set_ylabel('Scale Overhead (%)')
ax.set_title('Memory Overhead vs Group Size')
ax.set_xscale('log', base=2)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey insight: Smaller group sizes = better quality but more overhead.")
print("Common choices: 128 (balanced) or 32 (higher quality)")

---

## Part 3: Quantizing a Real Model with auto-gptq

In [ ]:
# Model selection for GPTQ quantization
from transformers import AutoModelForCausalLM, AutoTokenizer

# Choose a model
# For learning: use a smaller model
# For production: use your target model

MODEL_NAME = "microsoft/phi-2"  # 2.7B - good for demo
# MODEL_NAME = "meta-llama/Llama-2-7b-hf"  # 7B - standard benchmark

print(f"Selected model: {MODEL_NAME}")

In [ ]:
# Helper functions
def get_gpu_memory():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e9
    return 0

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded. Vocab size: {len(tokenizer)}")

In [ ]:
# Prepare calibration data

def get_calibration_data(tokenizer, num_samples=128, seq_len=512):
    """Get calibration data for GPTQ quantization."""
    try:
        from datasets import load_dataset
        
        print("Loading calibration data from C4...")
        dataset = load_dataset('allenai/c4', 'en', split='train', streaming=True)
        
        calibration_data = []
        for i, sample in enumerate(dataset):
            if i >= num_samples:
                break
            text = sample['text']
            if len(text) > 100:
                calibration_data.append(text)
        
    except Exception as e:
        print(f"Could not load C4: {e}")
        print("Using synthetic calibration data...")
        
        calibration_data = [
            "Machine learning models can be compressed through quantization techniques. "
            "This reduces memory usage and increases inference speed while maintaining accuracy.",
        ] * num_samples
    
    print(f"Prepared {len(calibration_data)} calibration samples")
    return calibration_data


calibration_data = get_calibration_data(tokenizer, num_samples=64)

In [ ]:
# Quantize with auto-gptq

if HAS_AUTOGPTQ:
    print("\nQuantizing with GPTQ...")
    print("=" * 60)
    
    # Configuration
    quantize_config = BaseQuantizeConfig(
        bits=4,              # 4-bit quantization
        group_size=128,      # Group size (try 32, 64, 128)
        desc_act=True,       # Activation order (improves quality)
    )
    
    print(f"Config: {quantize_config.bits}-bit, group_size={quantize_config.group_size}")
    
    # Load model for quantization
    print(f"\nLoading model for quantization...")
    start_time = time.time()
    
    model = AutoGPTQForCausalLM.from_pretrained(
        MODEL_NAME,
        quantize_config=quantize_config,
        trust_remote_code=True,
    )
    
    print(f"Model loaded in {time.time() - start_time:.1f}s")
    print(f"GPU memory: {get_gpu_memory():.2f} GB")
    
    # Quantize
    print(f"\nRunning GPTQ quantization (this may take a while)...")
    start_time = time.time()
    
    model.quantize(calibration_data)
    
    print(f"Quantization complete in {time.time() - start_time:.1f}s")
    print(f"GPU memory: {get_gpu_memory():.2f} GB")
    
else:
    print("auto-gptq not available. Showing expected workflow:")
    print("""
    from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
    
    config = BaseQuantizeConfig(bits=4, group_size=128)
    model = AutoGPTQForCausalLM.from_pretrained(MODEL_NAME, quantize_config=config)
    model.quantize(calibration_data)
    model.save_quantized("./my-model-gptq-4bit")
    """)
    model = None

In [ ]:
# Test the quantized model

if model is not None:
    test_prompt = "The key to machine learning is"
    
    print(f"Testing quantized model...")
    print(f"Prompt: '{test_prompt}'")
    
    inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\nGenerated: {response}")

---

## Part 4: Loading Pre-Quantized Models from Hugging Face

You don't always need to quantize yourself! Many GPTQ models are available on Hugging Face.

In [ ]:
# Clean up previous model
if 'model' in dir() and model is not None:
    del model
clear_memory()

# Load a pre-quantized GPTQ model
# TheBloke has quantized most popular models!

GPTQ_MODEL = "TheBloke/Llama-2-7B-GPTQ"  # Pre-quantized Llama 2 7B

print(f"Loading pre-quantized model: {GPTQ_MODEL}")
print("This is much faster than quantizing yourself!")

if HAS_AUTOGPTQ:
    try:
        start_time = time.time()
        
        gptq_model = AutoGPTQForCausalLM.from_quantized(
            GPTQ_MODEL,
            device="cuda:0",
            trust_remote_code=True,
            use_safetensors=True,
        )
        
        gptq_tokenizer = AutoTokenizer.from_pretrained(GPTQ_MODEL)
        if gptq_tokenizer.pad_token is None:
            gptq_tokenizer.pad_token = gptq_tokenizer.eos_token
        
        print(f"\nLoaded in {time.time() - start_time:.1f}s")
        print(f"GPU memory: {get_gpu_memory():.2f} GB")
        
    except Exception as e:
        print(f"Could not load pre-quantized model: {e}")
        print("This may require Hugging Face authentication for gated models.")
        gptq_model = None
else:
    gptq_model = None
    print("\nauto-gptq required to load GPTQ models.")

In [ ]:
# Benchmark the GPTQ model

if gptq_model is not None:
    print("Benchmarking GPTQ model...")
    
    prompt = "Explain the theory of relativity in simple terms:"
    inputs = gptq_tokenizer(prompt, return_tensors="pt").to(gptq_model.device)
    
    # Warmup
    with torch.no_grad():
        _ = gptq_model.generate(**inputs, max_new_tokens=10, do_sample=False,
                                pad_token_id=gptq_tokenizer.pad_token_id)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    # Benchmark
    num_tokens = 50
    num_runs = 3
    times = []
    
    for _ in range(num_runs):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        start = time.perf_counter()
        
        with torch.no_grad():
            outputs = gptq_model.generate(
                **inputs,
                max_new_tokens=num_tokens,
                do_sample=False,
                pad_token_id=gptq_tokenizer.pad_token_id
            )
        
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        times.append(time.perf_counter() - start)
    
    avg_time = sum(times) / len(times)
    tokens_per_second = num_tokens / avg_time
    
    print(f"\nGPTQ Performance:")
    print(f"  Tokens/second: {tokens_per_second:.1f}")
    print(f"  Latency: {avg_time * 1000:.1f} ms for {num_tokens} tokens")
    print(f"  Memory: {get_gpu_memory():.2f} GB")
    
    # Show generation
    response = gptq_tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\nGenerated:\n{response}")

---

## Part 5: Comparing Group Sizes

Let's compare different GPTQ configurations.

In [ ]:
# Comparison of GPTQ variants (from TheBloke's models)

print("GPTQ Configuration Comparison")
print("=" * 70)
print("""
Based on TheBloke's benchmarks for Llama 2 7B:

| Config | Group Size | desc_act | MMLU | Memory | Speed |
|--------|------------|----------|------|--------|-------|
| High Quality | 32 | Yes | 45.8% | 4.3 GB | 1.0x |
| Balanced | 128 | Yes | 45.5% | 4.1 GB | 1.1x |
| Fast | 128 | No | 45.2% | 4.0 GB | 1.2x |
| FP16 baseline | - | - | 46.0% | 13.5 GB | 0.5x |

Key observations:
1. GPTQ retains ~99% of FP16 quality on MMLU
2. Smaller group_size = better quality, more memory
3. desc_act=True helps quality but slows inference
4. 3x memory reduction vs FP16!
""")

print("\nRecommendations:")
print("  - group_size=128: Best balance of quality and speed")
print("  - group_size=32: When quality is critical")
print("  - desc_act=True: Default, disable only for max speed")

---

## ✋ Try It Yourself

### Exercise 1: Quantize with Different Group Sizes

Quantize the same model with group_size 32, 64, 128 and compare:
1. Memory usage
2. Inference speed
3. Output quality (run the same prompt)

### Exercise 2: Find Pre-Quantized Models

Search Hugging Face for GPTQ versions of your favorite model.
Hint: Search "TheBloke <model-name> GPTQ"

In [ ]:
# Exercise 1: Compare group sizes

# TODO: Quantize with different group sizes
# TODO: Compare memory and speed

# Your code here...

---

## Common Mistakes

### Mistake 1: Using Wrong Calibration Data

```python
# Wrong: Using random text for calibration
calib_data = ["lorem ipsum dolor sit amet..."]

# Right: Use data similar to your deployment use case
calib_data = load_your_domain_specific_data()
```

### Mistake 2: Ignoring desc_act

```python
# Wrong: Always using desc_act=False for speed
config = BaseQuantizeConfig(bits=4, desc_act=False)

# Right: Only disable if you've verified quality is acceptable
config = BaseQuantizeConfig(bits=4, desc_act=True)  # Default
# Run quality evaluation before disabling!
```

### Mistake 3: Not Using Pre-Quantized Models

```python
# Wrong: Always quantizing yourself
model.quantize(data)  # Takes hours!

# Right: Check if a pre-quantized version exists
# Search: https://huggingface.co/TheBloke
model = AutoGPTQForCausalLM.from_quantized("TheBloke/Model-GPTQ")
```

---

## Checkpoint

You've learned:

- **GPTQ algorithm**: Hessian-based quantization with error compensation
- **Group sizes**: Smaller = better quality, more overhead
- **Pre-quantized models**: TheBloke has 1000+ models ready to use!
- **Memory savings**: 3x reduction vs FP16, <1% quality loss

---

## Further Reading

- [GPTQ Paper](https://arxiv.org/abs/2210.17323)
- [auto-gptq GitHub](https://github.com/PanQiWei/AutoGPTQ)
- [TheBloke's Models](https://huggingface.co/TheBloke)

---

## Cleanup

In [ ]:
# Clean up
if 'gptq_model' in dir() and gptq_model is not None:
    del gptq_model
if 'model' in dir() and model is not None:
    del model

clear_memory()
print("Notebook complete! Ready for Lab 3.2.5: AWQ Quantization")

---

## Next Steps

In the next notebook, we'll explore **AWQ Quantization** - activation-aware quantization that often outperforms GPTQ!

➡️ Continue to: [Lab 3.2.5: AWQ Quantization](lab-3.2.5-awq-quantization.ipynb)